# Transform Data

Text

## Import Packages

In [1]:
import pandas as pd
from pathlib import Path
from scipy.constants import convert_temperature

## Define Functions

In [15]:
def load_climate_dataframe(filepath):
    """
    Load and preprocess climate data from a Parquet file.
    
    Parameters:
    - filepath: The file path to the climate data Parquet file.
    
    Returns:
    - DataFrame containing preprocessed climate data.
    """
    df = pd.read_parquet(filepath)
    
    # Filter to include only rows with projections
    if 'tsmin_df' in df.columns:
        df = df.loc[df['tsmin_df'] > 0]
    else:
        df = df.loc[df['tasmin'] > 0]
    
    # Create a new column
    df['model_scenario'] = df['model'] + '-' + df['scenario']
    
    # Round latitude and longitude to the nearest 0.5 degrees
    df['lat'] = (df['lat'] * 2).round() / 2
    df['lon'] = (df['lon'] * 2).round() / 2
    
    # Convert time to datetime format
    df['time'] = pd.to_datetime(df['time']).dt.date
    
    # Convert temperature from Kelvin to Fahrenheit
    if 'tsmin_df' in df.columns:
        df['fahrenheit'] = convert_temperature(df['tsmin_df'], 'kelvin', 'fahrenheit')
    else:
        df['fahrenheit'] = convert_temperature(df['tasmin'], 'kelvin', 'fahrenheit')
    
    # Pivot the DataFrame to reorganize data based on 'lat', 'lon', 'time'
    return df.pivot_table(index=['lat', 'lon', 'time'], columns='model_scenario', values='fahrenheit', aggfunc='min').reset_index()


In [4]:
def load_weather_dataframe(filepath):
    """
    Load and preprocess weather data from a CSV file.
    
    Parameters:
    - filepath: The file path to the weather data CSV file.
    
    Returns:
    - DataFrame containing preprocessed weather data.
    """
    # Load weather data from a CSV file
    df = pd.read_csv(filepath)
    
    # Round latitude and longitude to the nearest 0.5 degrees
    df['latitude'] = (df['latitude'] * 2).round() / 2
    df['longitude'] = (df['longitude'] * 2).round() / 2
    
    # Convert time to datetime format
    df['time'] = pd.to_datetime(df['time']).dt.date
    
    # Convert temperature from Celsius to Fahrenheit
    df['fahrenheit'] = convert_temperature(df['temperature'], 'celsius', 'fahrenheit')
    
    # Group by 'latitude', 'longitude', 'time' and calculate the minimum temperature
    return df.groupby(['latitude', 'longitude', 'time'], as_index=False).min()


In [5]:
def merge_dataframes(df_climate, df_weather):
    """
    Merge preprocessed climate and weather data DataFrames.
    
    Parameters:
    - df_climate: Preprocessed climate data.
    - df_weather: Preprocessed weather data.
    
    Returns:
    - DataFrame resulting from the merge of climate and weather data.
    """
    # Merge climate and weather data
    df = pd.merge(df_climate, df_weather, how='left', 
                  left_on=['lat', 'lon', 'time'], 
                  right_on=['latitude', 'longitude', 'time'])
    
    # Drop rows with any missing values resulting from the merge
    return df.dropna().drop(['latitude', 'longitude', 'time', 'temperature'], axis=1)
    

In [6]:
def save_dataframe(folderpath, filename, dataframe):
    
    # Create a Path object for folder_path to ensure correct path manipulation
    folder = Path(folderpath)

    # Combine the folder path and file name to create the full path to the file
    filepath = folder / 'processed' / filename
    
    dataframe.to_parquet(filepath, compression='gzip')
    

## Execute Functions

In [7]:
df_climate = load_climate_dataframe('../data/raw/train_east_tasmin_CMIP6_projections.parquet.gzip')
df_weather = load_weather_dataframe(weather_filepath)
df_train = merge_dataframes(df_climate, df_weather)

In [10]:
df_train.sample(5)

,lat,lon,ACCESS-CM2-ssp126,ACCESS-CM2-ssp245,ACCESS-CM2-ssp370,ACCESS-CM2-ssp585,ACCESS-ESM-ssp126,ACCESS-ESM-ssp245,ACCESS-ESM-ssp370,ACCESS-ESM-ssp585,...,INM-CM5-0-ssp585,KACE-1-0-G-ssp126,KACE-1-0-G-ssp245,KACE-1-0-G-ssp370,KACE-1-0-G-ssp585,MIROC-ES2L-ssp126,MIROC-ES2L-ssp245,MIROC-ES2L-ssp370,MIROC-ES2L-ssp585,fahrenheit
6375,35.5,-83.5,68.739620,52.199888,65.062040,70.030400,45.230666,59.959580,51.670688,58.539200,...,64.927400,66.420644,54.731804,50.271296,66.711686,57.029036,61.763216,63.010220,60.619424,61.411993
22943,36.0,-82.0,53.935790,19.672520,41.850500,33.140120,57.757154,19.055948,56.405012,43.532330,...,20.939504,33.376190,38.801660,40.334054,45.987674,51.055160,39.063092,50.926694,35.282480,40.622015
9830,35.5,-83.0,46.722650,26.146976,24.155474,25.927844,44.205800,11.208560,39.734960,32.132588,...,28.787540,31.358714,22.656560,24.895904,30.890804,55.999184,26.490128,33.562850,22.886060,34.088007
18826,36.0,-82.5,64.564016,43.181870,56.016320,50.534312,63.988052,60.524006,59.699606,60.585584,...,56.683220,61.803500,64.584230,56.438474,58.280108,52.142108,56.527844,57.662906,50.158580,61.430011
12324,35.5,-82.5,43.473920,63.002696,45.941720,24.905354,52.779920,42.300446,56.005394,57.892352,...,42.736874,56.861672,53.224430,53.016836,55.773410,41.872748,52.425554,41.003960,56.505020,58.298029


In [12]:
save_dataframe('../data/', 'df_easternmountain_train.parquet.gzip', df_train)

In [16]:
df_climate = load_climate_dataframe('../data/raw/val_east_tasmin_CMIP6_projections.parquet.gzip')
df_validate = merge_dataframes(df_climate, df_weather)

In [17]:
df_validate

,lat,lon,ACCESS-CM2-ssp126,ACCESS-CM2-ssp245,ACCESS-CM2-ssp370,ACCESS-CM2-ssp585,ACCESS-ESM-ssp126,ACCESS-ESM-ssp245,ACCESS-ESM-ssp370,ACCESS-ESM-ssp585,...,NorESM2-MM-ssp585,TaiESM1-ssp126,TaiESM1-ssp245,TaiESM1-ssp370,TaiESM1-ssp585,UKESM1-0-L-ssp126,UKESM1-0-L-ssp245,UKESM1-0-L-ssp370,UKESM1-0-L-ssp585,fahrenheit
0,35.0,-83.5,26.774726,28.582088,26.661920,29.723954,37.483484,30.601976,16.788434,32.676854,...,18.337712,50.748440,22.381340,49.895600,27.827276,-4.636642,26.445632,24.074114,23.796176,55.213068
1,35.0,-83.5,25.265534,27.326192,36.798764,27.884066,34.068488,27.412106,16.820960,22.415180,...,18.415724,43.746296,17.023154,54.175820,36.244580,23.193788,36.110300,9.892040,30.685586,53.199109
2,35.0,-83.5,36.079376,26.376980,36.802004,18.038786,21.980300,23.613512,21.808634,13.756208,...,32.811260,24.381680,17.267360,55.249808,27.594086,17.069288,43.619288,21.256844,37.480820,27.111249
3,35.0,-83.5,27.817430,24.001556,38.790320,11.067854,20.232374,15.316160,26.992940,13.673192,...,18.792140,19.589180,22.391348,55.863392,21.637346,17.205692,28.869386,18.764240,39.697700,21.894831
4,35.0,-83.5,34.506356,34.869668,38.328800,25.394792,27.519980,10.143680,23.199332,10.219874,...,22.088120,35.319020,27.354470,31.446824,26.382020,18.452588,29.055164,9.045860,36.269888,29.423761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14960,39.5,-78.0,7.496528,28.342814,32.319752,7.795184,40.224794,24.370646,17.610764,29.023736,...,36.366674,27.381290,32.786960,16.649672,47.141132,17.068820,40.405460,32.198000,40.598996,24.710846
14961,39.5,-78.0,8.342420,36.567284,39.352370,17.943368,44.050640,24.478808,20.408432,26.007116,...,33.957374,25.048166,36.881096,23.345420,32.527994,8.868596,39.229214,28.158800,42.043586,21.739703
14962,39.5,-78.0,20.740874,38.998940,35.563244,24.481940,32.628380,28.494320,19.044410,27.626180,...,37.612526,27.393854,33.640520,47.724080,31.885070,3.126812,31.361072,32.524808,37.363244,24.236676
14963,39.5,-78.0,10.904720,37.964912,31.989668,24.270440,29.817770,17.992940,8.796650,33.930608,...,36.095972,26.696012,43.813706,60.314900,41.120726,1.730480,33.137960,26.049956,28.385708,33.940570
